In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np

encoded_password = "Bsa1986%40%21"
DB_HOST = "127.0.0.1:3306"


DB_NAME = "bengaluru_traffic" 
engine = create_engine(f"mysql+pymysql://root:{encoded_password}@{DB_HOST}/{DB_NAME}")

query_area = """
SELECT
    road,
    area,
    COUNT(*) AS records,
    ROUND(AVG(congestion_score), 3) AS avg_congestion,
    ROUND(AVG(avg_speed), 2) AS avg_speed,
    ROUND(AVG(traffic_volume), 0) AS avg_volume,
    ROUND(AVG(environmental_impact), 2) AS avg_env_impact
FROM traffic_cleaned_geo
GROUP BY road, area
HAVING COUNT(*) >= 100  -- optional: ensure enough data points
ORDER BY avg_congestion DESC
"""

df_corridor = pd.read_sql(query_area, engine)
df_corridor

In [ ]:
#Corridor-Level Analysis

plt.figure(figsize=(14,7))

plt.barh((df_corridor['road'] + " (" + df_corridor['area'] + ")"), df_corridor['avg_congestion'], color='indianred')


plt.title("Average Congestion Score by Road", fontsize=16, weight='bold')
plt.xlabel("Average Congestion Score", fontsize=13)
plt.ylabel("Road (Area)", fontsize=13)

# Add subtle gridlines
plt.grid(axis='x', linestyle='--', alpha=0.75)

# Annotate values for clarity
for index, value in enumerate(df_corridor['avg_congestion']):
    plt.text(value+0.005, index, f'{value}',fontsize=10, ha='left') #small offset for better readability

# Highest congestion at the top
plt.gca().invert_yaxis()    
plt.tight_layout()
plt.show()

#Ques: Why horizontal Bar chart is used?
#Ans: “A horizontal bar chart was chosen to clearly compare congestion levels across corridors with long categorical labels.”

In [ ]:
#Impact of weather on Congestion:

query="""
SELECT
    weather,
    COUNT(*) AS records,
    ROUND(AVG(congestion_score), 3) AS avg_congestion,
    ROUND(STDDEV_SAMP(congestion_score),2) as volatility_data,
    ROUND(STDDEV_SAMP(congestion_score) / AVG(congestion_score),2) as coeff_of_variation,
    ROUND(AVG(avg_speed), 2) AS avg_speed,
    ROUND(AVG(traffic_volume), 0) AS avg_volume
FROM traffic_cleaned_geo
GROUP BY weather
ORDER BY avg_congestion DESC;
"""

df_weather=pd.read_sql_query(query,engine)
df_weather

In [ ]:
# x-axis
weather = df_weather['weather']

#Working with Categorical Data
x = np.arange(len(weather))
width = 0.25


plt.figure(figsize=(14,6)) 

plt.bar(x - width, df_weather['avg_congestion'], width, label='Avg Congestion', color='indianred')
plt.bar(x, df_weather['volatility_data'], width, label='Volatility (Std Dev)', color='steelblue')
plt.bar(x + width, df_weather['coeff_of_variation'], width, label='Coeff of Variation (%)', color='darkgreen')

plt.xticks(x, weather, fontsize=11,rotation=45) #Changing the labels along the x-axis

plt.xlabel("Weather Condition", fontsize=13)
plt.ylabel("Metric Value", fontsize=13)
plt.title("Impact of Weather on Congestion: Severity vs Volatility", fontsize=16, weight='bold')

plt.legend(fontsize=12) 
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Corridor Safety Correlation with Congestion-Road Capacity Utilization:
query="""
SELECT
    road,
    area,
    COUNT(*) as total_records,
    SUM(incident_reports) as total_incidents,
    ROUND(1.0 * SUM(incident_reports)/ COUNT(*),2) AS incident_per_records,
    ROUND(AVG(congestion_score), 3) AS avg_congestion,
    ROUND(AVG(traffic_volume),2) as avg_volume,
    ROUND(AVG(road_capacity_utilization),2) as road_capacity,
    ROUND(STDDEV_SAMP(road_capacity_utilization),2) as volatility_capacity
FROM traffic_cleaned_geo
GROUP BY road, area
ORDER BY incident_per_records DESC;

"""
df_safety=pd.read_sql_query(query,engine)
df_safety

In [ ]:
#A Scatterplot (Bubble Chart)
plt.figure(figsize=(12,8))

scatter = plt.scatter(
    df_safety['incident_per_records'],
    df_safety['avg_congestion'],
    s=df_safety['avg_volume'] * 0.25,   # bubble size scaling
    c=df_safety['road_capacity'],              
    cmap='coolwarm',
    alpha=0.7,
    edgecolors='black')

plt.xlabel("Incidents per Record (Normalized Incident Frequency)", fontsize=13)
plt.ylabel("Average Congestion Score", fontsize=13)
plt.title("Relationship Between Incident Frequency, Congestion, and Capacity (Road-Level)", fontsize=16, weight='bold')

# Color bar for speed
cbar = plt.colorbar(scatter)
cbar.set_label('Average Road Capacity', fontsize=12)

# Annotate road names
for i, row in df_safety.iterrows():
    plt.text(
        row['incident_per_records'] + 0.0005,
        row['avg_congestion'] + 0.0005,      #required offset for readability
        row['road'],
        fontsize=8
    )

plt.grid(alpha=0.4, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
#Environmental Impact and its relation with Congestion and Ped_cyclist mobility.
query="""
SELECT area,road,
	ROUND(AVG(traffic_volume),2) as avg_traffic_volume,
	ROUND(AVG(congestion_score),2) as avg_congestion,
    ROUND(AVG(ped_cycle_count),2) as avg_mobility,
    ROUND(AVG(road_capacity_utilization),2) as avg_capacity,
    ROUND(AVG(environmental_impact),2) as avg_env_impact,
    ROUND(STDDEV_SAMP(environmental_impact),2) as volatility_env
FROM traffic_cleaned_geo
GROUP BY area,road
ORDER BY avg_env_impact DESC;
"""
df_env=pd.read_sql_query(query,engine)
df_env

In [ ]:

import matplotlib.pyplot as plt

mob = df_env['avg_mobility']
mob_norm = (mob - mob.min()) / (mob.max() - mob.min())

bubble_size = (mob_norm ** 0.5) * 1500 + 100 #Scaling Work

plt.figure(figsize=(16,8))

scatter = plt.scatter(
    df_env['avg_congestion'],
    df_env['avg_env_impact'],
    s=bubble_size, 
    c=df_env['avg_capacity'],              
    cmap='coolwarm',
    alpha=0.7,
    edgecolors='black'
)
# Color bar for speed
cbar = plt.colorbar(scatter)
cbar.set_label('Average Road Capacity', fontsize=12)

# Reference lines (means)
mean_cong = df_env['avg_congestion'].mean()
mean_env = df_env['avg_env_impact'].mean()

plt.axvline(mean_cong, linestyle='--', color='grey', alpha=0.6) # these two lines made our scatter plot into quadrant-based scatter plot.
plt.axhline(mean_env, linestyle='--', color='grey', alpha=0.6)

plt.xlabel("Average Congestion Score", fontsize=13)
plt.ylabel("Average Env Impact", fontsize=13)
plt.title(
    "Congestion vs Environmental Impact\n(Bubble Size Indicates Pedestrian & Cyclist Mobility)",
    fontsize=15,
    weight='bold'
)

# Annotate points 
for i, row in df_env.iterrows():
        plt.text(
            row['avg_congestion'] + 0.005,
            row['avg_env_impact']+0.25,
            row['road'],
            fontsize=10
        )

plt.grid(alpha=0.4, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# Volatility Check Visuals Corridor wise:
query_vol = """
WITH daily AS (
    SELECT
        DATE(timestamp) AS dt,
        area,
        road,
        AVG(congestion_score) AS avg_congestion,
        AVG(traffic_volume) AS avg_volume
    FROM traffic_cleaned_geo
    GROUP BY DATE(timestamp), area, road
)
SELECT
    area,
    road,
    COUNT(*) AS days_observed,
    ROUND(AVG(avg_congestion), 3) AS mean_congestion,
    ROUND(AVG(avg_volume), 3) AS mean_volume,
    ROUND(STDDEV(avg_congestion), 3) AS stddev_congestion,
    ROUND(100.0 * STDDEV(avg_congestion) / NULLIF(AVG(avg_congestion),0), 3) AS coeff_of_variation
FROM daily
GROUP BY area, road
HAVING COUNT(*) >= 100
ORDER BY mean_congestion DESC, stddev_congestion DESC;
"""

df_quad = pd.read_sql(query_vol, engine)
df_quad

In [ ]:
# Compute dataset-driven thresholds
mean_cong = df_quad['mean_congestion'].mean()
cv = df_quad['coeff_of_variation'].mean()

plt.figure(figsize=(14,10))

plt.scatter(
    df_quad['mean_congestion'],
    df_quad['coeff_of_variation'],
    s=df_quad['mean_volume']*0.15,
    alpha=0.8,
    edgecolors='black'
)

# Draw quadrant lines
plt.axvline(mean_cong, linestyle='--', color='grey', linewidth=1)
plt.axhline(cv, linestyle='--', color='grey', linewidth=1)

# Annotate each corridor
for i, row in df_quad.iterrows():
    label = f"{row['road']} ({row['area']})"
    plt.text(
        row['mean_congestion'] + 0.002,
        row['coeff_of_variation'] + 0.15,
        label,
        fontsize=9
    )

plt.xlabel("Mean Congestion Score", fontsize=13)
plt.ylabel("Coefficient of Variation (%)", fontsize=13)
plt.title(
    "Quadrant-Based Segmentation of Corridors by Congestion Severity and Volatility",
    fontsize=16,
    weight='bold'
)
plt.grid(alpha=0.5, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
#HeatMap Comparision of Chronically Congested Corridor vs Event driven Congested Corridor:
import seaborn as sns  
query="""
SELECT
    DATE(timestamp) AS dt,
    area,
    ROUND(AVG(congestion_score), 3) AS avg_congestion
FROM traffic_cleaned_geo
WHERE area IN ('Koramangala', 'Whitefield','Hebbal','Indiranagar','Electronic City')
GROUP BY DATE(timestamp), area
ORDER BY area, dt;
"""
df_heat=pd.read_sql_query(query,engine)


df_heat['dt'] = pd.to_datetime(df_heat['dt'])
df_heat['avg_congestion'] = pd.to_numeric(df_heat['avg_congestion'])

df_area = df_heat[df_heat['area'].str.strip() == 'Koramangala'].copy()

# df_area has data in long format whereas for heatmap seaborne requires data to be in wide format.
pivot_c = df_area.pivot_table(
    index='area',
    columns='dt',
    values='avg_congestion'
)

# Plot
plt.figure(figsize=(18, 3))

sns.heatmap(
    pivot_c,
    cmap='Reds',
    cbar_kws={'label': 'Daily Avg Congestion Score'}
)

plt.title("Koramangala – Daily Congestion Heatmap (Chronic Corridor)", fontsize=14, weight='bold')
plt.xlabel("Date")
plt.ylabel("Corridor")

# Clean x-ticks
dates = pivot_c.columns
step = max(len(dates)//8, 1)

plt.xticks(
    ticks=range(0, len(dates), step),
    labels=[dates[i].strftime('%Y-%m') for i in range(0, len(dates), step)],
    rotation=45
)

plt.tight_layout()
plt.show()

In [ ]:
df_area_2 = df_heat[df_heat['area'] == 'Electronic City'].copy()

pivot_w = df_area_2.pivot_table(
    index='area',
    columns='dt',
    values='avg_congestion'
)

plt.figure(figsize=(18, 3))

sns.heatmap(
    pivot_w,
    cmap='Reds',
    cbar_kws={'label': 'Daily Avg Congestion Score'}
)

plt.title("Electronic City – Daily Congestion Heatmap (Peripheral Corridor)", fontsize=14, weight='bold')
plt.xlabel("Date")
plt.ylabel("Corridor")

dates = pivot_w.columns
step = max(len(dates)//8, 1)

plt.xticks(
    ticks=range(0, len(dates), step),
    labels=[dates[i].strftime('%Y-%m') for i in range(0, len(dates), step)],
    rotation=45
)
plt.tight_layout()
plt.show()

In [ ]:
#Performing Corridor vs Area level Congestion analysis for chronically congested corridors:
query="""
WITH daily as (
SELECT DATE(timestamp) as date,area,road,
	ROUND(AVG(congestion_score),2) as avg_congestion,
	ROUND(AVG(traffic_volume),2) as avg_traffic_volume
FROM traffic_cleaned_geo
GROUP BY DATE(timestamp),area,road),
area as (
SELECT date,area,
	ROUND(AVG(avg_congestion),2) as avg_congestion_area
FROM daily
GROUP BY date,area)

SELECT d.date,d.area,d.road,d.avg_congestion,a.avg_congestion_area,
	ROUND((d.avg_congestion-a.avg_congestion_area),2) as delta_congestion
FROM daily d JOIN area a ON d.area=a.area and d.date=a.date
ORDER BY delta_congestion DESC;
"""
import seaborn as sns


df_delta = pd.read_sql(query, engine)
df_delta['date'] = pd.to_datetime(df_delta['date'])

df_area = df_delta[df_delta['area'] == 'Koramangala'].copy()

#Creating a Uniform Date Range Index
full_dates = pd.date_range(
    start=df_area['date'].min(),
    end=df_area['date'].max(),
    freq='D'
)

corridors = df_area['road'].unique() 

pivot_dict = {}

for road in corridors:
    df_corridor = df_area[df_area['road'] == road]

    pivot = df_corridor.set_index('date')['delta_congestion']
    pivot = pivot.reindex(full_dates)  # THIS creates NaNs intentionally

    pivot_dict[road] = pivot
pivot_dict

In [ ]:
import numpy as np

n = len(pivot_dict)  # storing the number of corridors we have

fig, axes = plt.subplots(n, 1, figsize=(22, 2.5 * n), sharex=True) #sharex parameter: All subplots share the same x-axis scale and ticks


# Shared color scale
vmin = df_area['delta_congestion'].min()
vmax = df_area['delta_congestion'].max()

for ax, (road, series) in zip(axes, pivot_dict.items()):
    data = series.values.reshape(1, -1) #For heatmaps we need a wide format of data

    sns.heatmap(
        data,
        ax=ax,
        cmap='RdBu_r',
        center=0,
        vmin=vmin,
        vmax=vmax,
        cbar=False ,
        mask=np.isnan(data) #handle nan value for each corridor
    )
    ax.set_ylabel(road, rotation=0, labelpad=60, fontsize=11)
    ax.set_yticks([])

# Shared colorbar
sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])

cbar = fig.colorbar(sm, ax=axes, orientation='vertical')
cbar.set_label('Corridor Deviation from Area Avg Congestion')


#X-axis formatting
dates = full_dates
step = max(len(dates)//8, 1)

axes[-1].set_xticks(range(0, len(dates), step))
axes[-1].set_xticklabels(
    [dates[i].strftime('%Y-%m') for i in range(0, len(dates), step)],
    rotation=45
)

plt.suptitle(
    "Koramangala: Corridor-Level Deviation from Area Congestion Baseline",
    fontsize=16,
    weight='bold'
)
plt.show()